# What is about 

Preliminary look at breast cancer METABRIC dataset.

There are several version of that dataset on kaggle. 
Here we use data from https://www.kaggle.com/alexandervc/breast-cancer-omics-bulk-data
which contains both some clinical and OMICs data


In [ ]:
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

### Importing/installing libraries

In [ ]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

from importlib import reload  
import scipy.stats


In [ ]:
!pip install lifelines
from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
!pip install  --no-dependencies  git+https://github.com/j-bac/elpigraph-python.git
import elpigraph
!pip install trimap

import sys
sys.path.insert(0,'/kaggle/input/breast-cancer-omics-bulk-data/code/')# "/path/to/your/package_or_module")
print(sys.path)

from clintraj_qi import *
from clintraj_eltree import *
from clintraj_util import *
from clintraj_ml import *
from clintraj_optiscale import *

# Load data joining clinical and OMICs

In [ ]:
# load omics data
df1 = pd.read_csv('/kaggle/input/breast-cancer-omics-bulk-data/METABRIC.txt', sep = '\t', index_col = 0)
df1=df1.T
i1 = [s.replace('BRCA-METABRIC-S1-','') for s in df1.index ]
#print('number of common ids:', len(set(i2) & set(df1.index) ) )
df1.index = i1
df1
# load clinical data
df2 = pd.read_csv('/kaggle/input/breast-cancer-omics-bulk-data/METABRIC_clinical.txt', sep = '\t')#, index_col = 0)
df2 = df2.set_index('Patient ID')
df2
df = df2.join(df1, how = 'inner')
print('Joined data shape', df.shape)
df
m = df['Relapse Free Status'].notnull()
print( m.sum() )
df = df[m].copy()
df['Relapse Free Status'] = df['Relapse Free Status'].map({'0:Not Recurred':0,'1:Recurred':1 } )
print(df.shape)
display(df.head())

df_full = df.copy()
df = df.iloc[:,37:] # OMICS data only

# Clinical features

In [ ]:
print( df_full.columns[:37] )


# PCA plots

In [ ]:
f = 'Pam50 + Claudin-low subtype' #   'pam50_+_claudin-low_subtype'
vec4color = df_full[f]

X = df.values

from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
pca = PCA
r = pca().fit_transform(X = X )
plt.figure(figsize = (20,10))
sns.scatterplot( x=r[:,0], y=r[:,1], hue = vec4color )
plt.title('PCA for Omics data colored by Pam50 groups')
plt.show()

In [ ]:
X.shape

In [ ]:
for col in df_full.columns[2:37]:
    plt.figure(figsize = (20,5))
    sns.scatterplot( x=r[:,0], y=r[:,1], hue = df_full[col] )
    plt.title('PCA for Omics data colored by ' + col)
    plt.show()

# Selected correlations

In [ ]:
# FOXA1 in breast cancer is highly correlated with ERα+, GATA3+, and PR+ protein expression as well as endocrine signaling.
# https://en.wikipedia.org/wiki/FOXA1#Marker_in_breast_cancer

df[['FOXA1', 'ESR1','GATA3', 'PGR']].corr()

In [ ]:
cm = np.corrcoef(df.values.transpose() )

In [ ]:
cm.shape

In [ ]:
list(df.columns).index('C1S')

In [ ]:
s = pd.Series( cm[23793,:], index = df.columns)
s.sort_values()[:20]

In [ ]:
s.sort_values()[-20:]